In [6]:
import os
import sys
import re
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import openpyxl
import pandas as pd
import math
import itertools
import matplotlib.patches as patches
import seaborn as sns
from decimal import Decimal
from csv import reader
import csv
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.utilities.distribution import MultiprocessingDistributor

def readGPU_power_C(fileName, outFile):
 
    data= []
    time_units =0.1

    with open(fileName) as f:
        next(f)
        t=0
        for line in f:
            temp = line.split(',')
            temp_row = [float(i.split()[0]) for i in temp]
            temp_row.insert(0, t)
            data.append(temp_row)
            t+=time_units

    columns= ["time", "power", "memory Used", "u_gpu", "u_memory"]
    df = pd.DataFrame(data=data, columns=columns)
    df.to_csv(outFile,index=False)
    print(outFile)
    
def get_app_list(fileName):
    """get metrics list

    Arguments:
        fileName {string} -- the file name of application file with absolute path

    Returns:
        app {list} -- the app  list
    """
    apps = []
    with open(fileName, "r") as f:
        for line in f.readlines():
            if not line.startswith('#'):
                words = line.strip().split(',')
                app = words[0].strip()
                app_num = words[1].strip()
                apps.append(app+app_num)
        f.close()
    return apps

def Combine_data(pathfolder, app_list, outFile):
    writer = pd.ExcelWriter(outFile,engine='xlsxwriter')
    data = []    
    i = 0
    for app in app_list:
        fileName =  os.path.join(pathfolder, "{}-ori.csv".format(app))
        data_temp = pd.read_csv(fileName)
        data_temp["app"] = app
        data_temp["ID"] = i
        data.append(data_temp)
        i+=1
        if  i%200 == 0:        
            outData = pd.concat(data)    
            outData.to_excel(writer, sheet_name="{}-{}".format(i-199, i), index=False)
            data = []
        
        elif i==len(app_list):
            if i >= 200:
                x = i -199
            else:
                x=1
            outData = pd.concat(data)    
            outData.to_excel(writer, sheet_name="{}-{}".format(x, i), index=False)
    writer.save()

   
    
def readGPU_power(pathfolder, app_list):
    for app in app_list:
        fileName = os.path.join(pathfolder, "{}.pwr.txt".format(app))
        outFile = os.path.join(pathfolder, "{}.pwr.csv".format(app))
        readGPU_power_C(fileName, outFile)

def process_Data():
    category = "mybench"
    arch ="k40"
    #for arch in ["K40", "V100"]:
    pathfolder = r"/home/pzou/projects/Power_Signature/results/%s/%s/power-combine"%(category, arch)
    app_list = get_app_list("/home/pzou/projects/Power_Signature/Scripts/applications_%s.csv"%(category))
    #readGPU_power(pathfolder, app_list)
    outFile = "power-combine-%s-%s.xlsx"%(category, arch)
    print("start")
    Combine_data(pathfolder, app_list, outFile)
    print("Done")
    
def extractFeature(dataName, outName):

    Distributor = MultiprocessingDistributor(n_workers=16,
                                         disable_progressbar=False,
                                         progressbar_title="Feature Extraction")

    xlData = pd.ExcelFile(dataName)
    data = []
    for sheets in xlData.sheet_names:
        dataX = xlData.parse( sheet_name=sheets) 
        data.append(dataX)
    data = pd.concat(data)    
    dataX = data.drop(columns=["app"])
    extracted_features = extract_features(dataX, column_id="ID", column_sort="start", distributor=Distributor)
    extracted_features.to_csv(outName, index=False)

arch = "p100"
category="risky"
tFtype = "mem_trace-combine"
dataName = "{}-{}-{}.xlsx".format(tFtype, category,arch)
pathfolder = r"/home/pzou/projects/Power_Signature/results/%s/%s/mem_trace-combine"%(category, arch)
app_list = get_app_list("/home/pzou/projects/Power_Signature/Scripts/applications-mem_%s.csv"%(category))

#outFile = "power-combine-%s-%s.xlsx"%(category, arch)

print("start")
#Combine_data(pathfolder, app_list, dataName)
print("done combine")
featureName = "{}-feature-{}-{}.csv".format(tFtype, category,arch)

extractFeature(dataName, featureName)
print("done feature")

start
done combine



Feature Extraction:   0%|          | 0/77 [00:00<?, ?it/s]/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1515: RuntimeWarning: invalid value encountered in true_divide
  similarity_ratio = A / B
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1516: RuntimeWarning: divide by zero encountered in log
  se = -1 * np.log(similarity_ratio)
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1515: RuntimeWarning: invalid value encountered in true_divide
  similarity_ratio = A / B
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1516: RuntimeWarning: divide by zero encountered in log
  se = -1 * np.log(similarity_ratio)
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1515: RuntimeWarning: invalid value encountered in true_divide
  similarity_ratio = A / B
/soft

/software/anaconda3/5.1.0/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:660: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/software/anaconda3/5.1.0/lib/python3.6/site-packages/statsmodels/tsa/stattools.py:772: RuntimeWarning: invalid value encountered in double_scalars
  phi[1, 1] = sxx_m[1] / sxx_m[0]
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1515: RuntimeWarning: invalid value encountered in true_divide
  similarity_ratio = A / B
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:989: RuntimeWarning: invalid value encountered in double_scalars
  return y.dot(np.arange(len(y))**moment) / y.sum()
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1516: RuntimeWarning: divide by zero encountered in log
  se = -1 * np.log(similarity_ratio)
/home/pzou/.loca

/software/anaconda3/5.1.0/lib/python3.6/site-packages/statsmodels/base/model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/software/anaconda3/5.1.0/lib/python3.6/site-packages/statsmodels/base/model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/software/anaconda3/5.1.0/lib/python3.6/site-packages/statsmodels/tsa/stattools.py:772: RuntimeWarning: invalid value encountered in double_scalars
  phi[1, 1] = sxx_m[1] / sxx_m[0]
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1880: RuntimeWarning: invalid value encountered in double_scalars
  res_data.append(np.sum(np.array_split(x, num_segments)[segment_focus] ** 2.0)/full_series_energy)
/software/anaconda3/5.1.0/lib/python3.6/site-packages/statsmodels/tsa/stattools.py:772: RuntimeWarning: invalid value encountered in double_scalars
  phi[1, 1] = sxx_m[1] / sxx_m[0]
/home/pzou/.local/lib/py

/software/anaconda3/5.1.0/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/software/anaconda3/5.1.0/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1515: RuntimeWarning: invalid value encountered in true_divide
  similarity_ratio = A / B
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1516: RuntimeWarning: divide by zero encountered in log
  se = -1 * np.log(similarity_ratio)
/software/anaconda3/5.1.0/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1562: RuntimeWarning: invalid

...

/software/anaconda3/5.1.0/lib/python3.6/site-packages/statsmodels/tsa/stattools.py:772: RuntimeWarning: invalid value encountered in double_scalars
  phi[1, 1] = sxx_m[1] / sxx_m[0]
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:989: RuntimeWarning: invalid value encountered in double_scalars
  return y.dot(np.arange(len(y))**moment) / y.sum()
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1880: RuntimeWarning: invalid value encountered in double_scalars
  res_data.append(np.sum(np.array_split(x, num_segments)[segment_focus] ** 2.0)/full_series_energy)
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1562: RuntimeWarning: invalid value encountered in double_scalars
  return sum_product / ((len(x) - lag) * v)
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1515: RuntimeWarning: divide by zero enco

/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1515: RuntimeWarning: divide by zero encountered in true_divide
  similarity_ratio = A / B
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:148: RankWarning: Polyfit may be poorly conditioned
  return np.polyfit(result.x_mean, result.y_mean, deg=m)
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:148: RankWarning: Polyfit may be poorly conditioned
  return np.polyfit(result.x_mean, result.y_mean, deg=m)
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1515: RuntimeWarning: invalid value encountered in true_divide
  similarity_ratio = A / B
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1516: RuntimeWarning: divide by zero encountered in log
  se = -1 * np.log(similarity_ratio)
/home/pzou/.local/lib/python

/software/anaconda3/5.1.0/lib/python3.6/site-packages/scipy/signal/_peak_finding.py:412: RuntimeWarning: divide by zero encountered in double_scalars
  snr = abs(cwt[line[0][0], line[1][0]] / noises[line[1][0]])
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1516: RuntimeWarning: divide by zero encountered in log
  se = -1 * np.log(similarity_ratio)
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:148: RankWarning: Polyfit may be poorly conditioned
  return np.polyfit(result.x_mean, result.y_mean, deg=m)
/software/anaconda3/5.1.0/lib/python3.6/site-packages/statsmodels/tsa/stattools.py:772: RuntimeWarning: invalid value encountered in double_scalars
  phi[1, 1] = sxx_m[1] / sxx_m[0]
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:148: RankWarning: Polyfit may be poorly conditioned
  return np.polyfit(result.x_mean, result.y_mean, deg=m)
/home/p

Feature Extraction:  64%|██████▎   | 49/77 [01:44<02:22,  5.09s/it]/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1515: RuntimeWarning: invalid value encountered in true_divide
  similarity_ratio = A / B
/home/pzou/.local/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1516: RuntimeWarning: divide by zero encountered in log
  se = -1 * np.log(similarity_ratio)
/software/anaconda3/5.1.0/lib/python3.6/site-packages/statsmodels/tsa/stattools.py:772: RuntimeWarning: invalid value encountered in double_scalars
  phi[1, 1] = sxx_m[1] / sxx_m[0]
/software/anaconda3/5.1.0/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:660: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/software/anaconda3/5.1.0/lib/python3.6/site-packages/statsmodels/base/model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.param

MemoryError: 